In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization
from isic_helper import compute_pauc, compute_auc

In [2]:
def blend_optimizer(oof_preds_df, oof_columns, folds, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 10.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights

In [3]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_ensemble"
fold_column = "gkf_fold"

boosting_model_names = ["xgb", "xgb", "lgb"]
boosting_versions = ["v1", "v4", "v6"]
boosting_modes = ["train", "train", "train"]
boosting_oof_columns = [f"oof_{model_name}_{version}" for model_name, version in zip(boosting_model_names, boosting_versions)]

cnn_model_names = ["efficientnet_b0", "efficientnet_b2"]
cnn_versions = ["v1", "v1"]
cnn_modes = ["trainbinary", "trainbinary"]
cnn_oof_columns = [f"oof_{model_name}_{version}" for model_name, version in zip(cnn_model_names, cnn_versions)]

model_names = boosting_model_names + cnn_model_names
versions = boosting_versions + cnn_versions
modes = boosting_modes + cnn_modes
oof_columns = boosting_oof_columns + cnn_oof_columns

paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    mode = modes[idx]
    model_identifier = f"{model_name}_{version}"
    oof_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_identifier}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}

all_folds = np.unique(oof_preds_df["fold"])
weights = blend_optimizer(
    oof_preds_df, boosting_oof_columns + cnn_oof_columns, all_folds,
    init_points=50, n_iter=100
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |    w4     |
-------------------------------------------------------------------------------------
| 1         | 0.1801    | 0.09359   | 4.991     | 1.134     | 0.4997    | 6.854     |
| 2         | 0.1822    | 4.87      | 8.977     | 6.475     | 8.97      | 7.211     |
| 3         | 0.1827    | 8.314     | 8.276     | 8.336     | 9.57      | 3.68      |
| 4         | 0.1814    | 4.948     | 3.395     | 6.194     | 9.775     | 0.9643    |
| 5         | 0.1817    | 7.442     | 2.925     | 2.987     | 7.525     | 0.1866    |
| 6         | 0.183     | 5.237     | 8.644     | 3.888     | 2.122     | 4.752     |
| 7         | 0.1833    | 5.647     | 3.494     | 9.759     | 0.3782    | 7.943     |
| 8         | 0.1829    | 3.579     | 7.48      | 9.145     | 3.727     | 9.649     |
| 9         | 0.1805    | 0.8139    | 0.4245    | 2.968     | 3.637     | 4.903     |
| 10        | 0.1832    | 6.685     | 6.734     | 5.72

In [4]:
weights

[3.03634383136314,
 4.299001604386792,
 7.66164388355334,
 0.7268463084793878,
 3.2888862981975295]

In [5]:
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")
print(f"CV PAUC RANGE: ({cv_ensemble_pauc_avg - cv_ensemble_pauc_std}, {cv_ensemble_pauc_avg + cv_ensemble_pauc_std})")

Model: xgb_v1 | Weightage: 3.03634383136314
Val AUC scores:
{'fold_1': 0.9859929805273405,
 'fold_2': 0.976488740334299,
 'fold_3': 0.9581711509902535,
 'fold_4': 0.9793895308931625,
 'fold_5': 0.9781104298718765}
Val PAUC scores:
{'fold_1': 0.1897381254082902,
 'fold_2': 0.18200898111048602,
 'fold_3': 0.16480497560245091,
 'fold_4': 0.18248617888208055,
 'fold_5': 0.1830007932325329}
CV AUC OOF: 0.970935752023771
CV PAUC OOF: 0.17590707654960594
CV AUC AVG: 0.9756305665233865
CV PAUC AVG: 0.18040781084716812
CV AUC STD: 0.009308349073795128
CV PAUC STD: 0.008295904471903799


Model: xgb_v4 | Weightage: 4.299001604386792
Val AUC scores:
{'fold_1': 0.9774012004442393,
 'fold_2': 0.9585113204613043,
 'fold_3': 0.9635616209706606,
 'fold_4': 0.9713384957132696,
 'fold_5': 0.9692148821814502}
Val PAUC scores:
{'fold_1': 0.18439364147339235,
 'fold_2': 0.16635088150424956,
 'fold_3': 0.17198002333678597,
 'fold_4': 0.174545119866219,
 'fold_5': 0.17352956799065142}
CV AUC OOF: 0.9478567231

In [6]:
oof_preds_df.pivot_table(index="target", values=oof_columns + [ensemble_column], aggfunc="mean")

,oof_efficientnet_b0_v1,oof_efficientnet_b2_v1,oof_ensemble,oof_lgb_v6,oof_xgb_v1,oof_xgb_v4
target,,,,,,
0,0.002347,0.000783,9.497688,0.011610,0.014133,0.014427
1,0.289894,0.211758,18.469353,0.445961,0.397983,0.329018


In [7]:
oof_preds_df[oof_columns + [ensemble_column]].corr()

,oof_xgb_v1,oof_xgb_v4,oof_lgb_v6,oof_efficientnet_b0_v1,oof_efficientnet_b2_v1,oof_ensemble
oof_xgb_v1,1.000000,0.768666,0.953573,0.517377,0.374600,0.372296
oof_xgb_v4,0.768666,1.000000,0.728958,0.530451,0.387978,0.324754
oof_lgb_v6,0.953573,0.728958,1.000000,0.544058,0.420514,0.372253
oof_efficientnet_b0_v1,0.517377,0.530451,0.544058,1.000000,0.783158,0.141748
oof_efficientnet_b2_v1,0.374600,0.387978,0.420514,0.783158,1.000000,0.080445
oof_ensemble,0.372296,0.324754,0.372253,0.141748,0.080445,1.000000
